## R-CNN

예전의 객체 인식 알고리즘들은 `슬라이딩 윈도우(sliding window)` 방식, 즉 일정한 크기를 가지는 윈도우(window)를 가지고 이미지의 모든 영역을 탐색하면서 객체를 검출해 내는 방식이었습니다. 하지만 알고리즘의 비효율성 때문에 많이 사용하지 않았으며, 현재는 선택적 탐색(selective search) 알고리즘을 적용한 후보 영역(region proposal)을 많이 사용합니다.

R-CNN(Region-based CNN)은 이미지 분류를 수행하는 CNN과 이미지에서 객체가 있을 만한 영역을 제안해 주는 후보 영역 알고리즘을 결합한 알고리즘입니다. R-CNN의 수행 과정은 다음 그림과 같습니다.

![](../Static/343_1.jpg)

* 이미지를 입력으로 받습니다.

* 2000개의 바운딩 박스(bounding box)를 선택적 탐색 알고리즘으로 추출한 후 잘라 내고, CNN 모델에 넣기 위해 같은 크기(227 x 227 px)로 통일합니다.

* 크기가 동일한 이미지 2000개에 각각 cnn 모델을 적용합니다.

* 각각 분류를 진행하여 결과를 도출합니다.

R-CNN은 성능이 뛰어나기는 하지만 다음과 같은 단점으로 크게 발전하지는 못했습니다.

1. 앞서 언급한 세 단계의 복잡한 학습 과정

2. 긴 학습 시간과 대용량 저장 공간

3. 객체 검출 속도 문제

이러한 문제를 해결하기 위해 Fast R-CNN이 생겼습니다.

### 공간 피라미드 풀링

기존 CNN 구조들은 모두 완전연결층을 위해 입력 이미지를 고정해야 했습니다. 그렇기 때문에 신경망을 통과시키려면 이미지를 고정된 크기로 자르거나(crop) 비율을 조정(warp)해야 했습니다. 하지만 이렇게 하면 물체의 일부분이 잘리거나 본래의 생김새와 달라지는 문제점이 있습니다. 이러한 문제를 해결하고자 `공간 피라미드 풀링(spatial pyramid pooling)`을 도입했습니다.

![](../Static/345.jpg)

즉, 공간 피라미드 풀링은 입력 이미지의 크기에 관계없이 합성곱층을 통과시키고, 완전연결층에 전달되기 전에 특성 맵들을 동일한 크기로 조절해 주는 풀링층을 적용하는 기법입니다. 입력 이미지의 크기를 조절하지 않고 합성곱층을 통과시키기 때문에 원본 이미지의 특징이 훼손되지 않는 특성 맵을 얻을 수 있습니다. 또한, 이미지 분류나 객체 인식 같은 여러 직업에 적용할 수 있다는 장점이 있습니다.



## Fast R-CNN

R-CNN은 바운딩 박스마다 CNN을 돌리고, 분류를 위한 긴 학습 시간이 문제였습니다. Fast R-CNN은 R-CNN의 속도 문제를 개선하려고 `Rol 풀링`을 도입했습니다. 즉, 선택적 탐색에서 찾은 바운딩 박스 정보가 CNN을 통과하면서 유지되도록 하고 최종 CNN 특성 맵으 풀링을 적용하여 완전연결층을 통과하도록 크기를 조정합니다. 이렇게 하면 바운딩 박스마다 CNN을 돌리는 시간을 단축할 수 있습니다.

![](../Static/346_1.jpg)

* Rol 풀링

Rol 풀링은 크기가 다른 특성 맵의 영역마다 스트라이드를 다르게 최대 풀링을 적용하여 결과값 크기를 동일하게 맞추는 방법입니다.

## Faster R-CNN

Faster R-CNN은 '더욱 빠른' 객체 인식을 수행하기 위한 네트워크입니다. 기존 Fast R-CNN 속도의 걸림돌이었던 후보 영역 생성을 CNN 내부 네트워크에서 진행할 수 있도록 설계했습니다. 즉, Faster R-CNN은 기존 Fast R-CNN에 `후보 영역 추출 네트워크(Region Proposal Network, RPN)`를 추가한 것이 핵심이라고 할 수 있습니다. Faster R-CNN에서는 외부의 느린 선택적 탐색(CPU로 계산) 대신 내부의 빠른 RPN(GPU로 계산)을 사용합니다.

RPN은 다음 그림과 같이 마지막 합성곱층 다음에 위치하고, 그 뒤에 Fast R-CNN과 마찬가지로 Rol 풀링과 분류기, 바운딩 박스 회귀가 위치합니다.

![](../Static/347_2.jpg)

후보 영역 추출 네트워크는 특성 맵 N x N 크기의 작은 윈도우 영역을 입력으로 받고, 해당 영역에 객체의 존재 유무 판단을 위해 이진 분류(binary classification)를 수행하는 작은 네트워크를 생성합니다. R-CNN, Fast R-CNN에서 사용되었던 바운딩 박스 회귀 또한 위치 보정(좌표점추론)을 위해 추가합니다. 또한, 하나의 특성 맵에서 모든 영역에 대한 객체의 존재 유무를 확인하기 위해서는 슬라이딩 윈도우 방식으로 앞서 설계한 작은 윈도우 영역(N x N 크기)을 이용하여 객체를 탐색합니다.

![](../Static/348.jpg)

하지만 후보 영역 추출 네트워크는 이미지에 존재하는 객체들의 크기와 비율이 다양하기 때문에 고정된 N x N 크기의 입력만으로 다양한 크기와 비율의 이미지를 수용하기 어려운 단점이 있습니다. 이러한 단점을 보완하기 위해 여러 크기와 비율의 레퍼런스 박스(reference box) k개를 미리 정의하고 각각의 슬라이딩 윈도우 위치마다 박스 k개를 출력하도록 설계하는데, 이 방식을 앵커(anchor)라고 합니다. 즉, 후보 영역 추출 네트워크의 출력 값은 모든 앵커 위치에 대해 각각 객체와 배경을 판단하는 2k개의 분류에 대한 출력과 x,y,w,h 위치 보정 값을 위한 4k개의 회귀 출력을 갖습니다. 예를 들어 특성 맵 크기가 w x h 라면 하나의 특성 맵에 앵커가 총 w x h x k개 존재합니다.

![](../Static/349.jpg)



